In [ ]:
!wget https://s3-ap-southeast-1.amazonaws.com/he-public-data/datasetb2d9982.zip

--2023-06-21 11:04:20--  https://s3-ap-southeast-1.amazonaws.com/he-public-data/datasetb2d9982.zip
Resolving s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)... 52.219.128.170
Connecting to s3-ap-southeast-1.amazonaws.com (s3-ap-southeast-1.amazonaws.com)|52.219.128.170|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 895569552 (854M) [binary/octet-stream]
Saving to: ‘datasetb2d9982.zip’

datasetb2d9982.zip  100%[===================>] 854.08M  16.8MB/s    in 54s     

2023-06-21 11:05:15 (15.9 MB/s) - ‘datasetb2d9982.zip’ saved [895569552/895569552]



In [ ]:
import zipfile
import os
# Define the path to the ZIP file
zip_file_path = 'datasetb2d9982.zip'

# Extract the contents of the ZIP file to a directory
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()

# Set the file paths for the extracted dataset
base_dir = 'dataset'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')
sample_submission_dir = os.path.join(base_dir, 'sample_submission')


In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the training and testing data into Pandas dataframes
train_df = pd.read_csv("dataset/train.csv")
test_df = pd.read_csv("dataset/test.csv")



In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2249698 entries, 0 to 2249697
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   PRODUCT_ID       int64  
 1   TITLE            object 
 2   BULLET_POINTS    object 
 3   DESCRIPTION      object 
 4   PRODUCT_TYPE_ID  int64  
 5   PRODUCT_LENGTH   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 103.0+ MB


In [ ]:
train_df.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000


In [ ]:
train_df.isna().sum()

PRODUCT_ID               0
TITLE                   12
BULLET_POINTS       837364
DESCRIPTION        1157381
PRODUCT_TYPE_ID          0
PRODUCT_LENGTH           0
dtype: int64

In [ ]:
train_df['BULLET_POINTS'].fillna('None', inplace=True)

In [ ]:
train_df.isna().sum()

PRODUCT_ID               0
TITLE                   12
BULLET_POINTS            0
DESCRIPTION        1157381
PRODUCT_TYPE_ID          0
PRODUCT_LENGTH           0
dtype: int64

In [ ]:
train_df['DESCRIPTION'].fillna('None', inplace=True)

In [ ]:
train_df.isna().sum()

PRODUCT_ID          0
TITLE              12
BULLET_POINTS       0
DESCRIPTION         0
PRODUCT_TYPE_ID     0
PRODUCT_LENGTH      0
dtype: int64

In [ ]:
train_df = train_df[train_df['TITLE'].notna()]



In [ ]:
train_df.isna().sum()


PRODUCT_ID         0
TITLE              0
BULLET_POINTS      0
DESCRIPTION        0
PRODUCT_TYPE_ID    0
PRODUCT_LENGTH     0
dtype: int64

In [ ]:
test_df.isna().sum()

PRODUCT_ID              0
TITLE                   5
BULLET_POINTS      275922
DESCRIPTION        380001
PRODUCT_TYPE_ID         0
dtype: int64

In [ ]:
test_df['BULLET_POINTS'].fillna('None', inplace=True)

In [ ]:
test_df.isna().sum()

PRODUCT_ID              0
TITLE                   5
BULLET_POINTS           0
DESCRIPTION        380001
PRODUCT_TYPE_ID         0
dtype: int64

In [ ]:
test_df['DESCRIPTION'].fillna('None', inplace=True)

In [ ]:
test_df.isna().sum()

PRODUCT_ID         0
TITLE              5
BULLET_POINTS      0
DESCRIPTION        0
PRODUCT_TYPE_ID    0
dtype: int64

In [ ]:
test_df = test_df[test_df['TITLE'].notna()]

In [ ]:
test_df.isna().sum()

PRODUCT_ID         0
TITLE              0
BULLET_POINTS      0
DESCRIPTION        0
PRODUCT_TYPE_ID    0
dtype: int64

In [ ]:
training_sentences = train_df["PRODUCT_ID"].astype(str).tolist() + train_df["PRODUCT_LENGTH"].astype(str).tolist()
testing_sentences = test_df["PRODUCT_ID"].astype(str).tolist() + train_df["PRODUCT_LENGTH"].astype(str).tolist()


In [ ]:
import numpy as np

# Extract the labels for the training data
labels = train_df["PRODUCT_ID"].astype(str).tolist()
# Extract the labels for the testing data
validation_labels = test_df["PRODUCT_ID"].astype(str).tolist()

validation_labels = np.array(validation_labels, dtype=np.float32)
labels = np.array(labels, dtype=np.float32)



# Initialize tokenizer with a maximum vocabulary size of 10000
tokenizer = Tokenizer(num_words=100000, oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)

# Convert sentences to sequences of integers using the word index
training_sequences = tokenizer.texts_to_sequences(training_sentences)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

# Pad sequences to a fixed length
max_length = 100000
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding='post', truncating='post')
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding='post', truncating='post')

# Get the word index with the OOV token
word_index = tokenizer.word_index

In [ ]:
import tensorflow as tf

# Define the input layer
input_layer = tf.keras.Input(shape=(126,))

# Add hidden layers
hidden1 = tf.keras.layers.Dense(64, activation='relu')(input_layer)
hidden2 = tf.keras.layers.Dense(32, activation='relu')(hidden1)

# Add output layer for regression
output_layer = tf.keras.layers.Dense(1)(hidden2)

# Create the model
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

# Print the model summary
model.summary()

In [ ]:
import numpy as np
validation_labels = np.array(validation_labels, dtype=np.float32)
labels = np.array(labels, dtype=np.float32)


In [ ]:
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])

In [ ]:
history = model.fit(training_padded, labels, epochs=1, validation_data=(testing_padded, validation_labels ), verbose=2)

In [ ]:
# Evaluate the model on the test data
# Evaluate the model on the test data
test_loss, test_mae = model.evaluate(testing_padded, test_df["PRODUCT_LENGTH"])


# Generate predictions on the test data
predictions = model.predict(testing_padded)
print(predictions)

In [ ]:
# Create a dataframe with the predictions
submission_df = pd.DataFrame({"PRODUCT_ID": test_df["PRODUCT_ID"], "PRODUCT_LENGTH": predictions.flatten()})

# Format the dataframe according to the sample submission file
submission_df = submission_df[["PRODUCT_ID", "PRODUCT_LENGTH"]]
submission_df.to_csv("submission.csv", index=False)